# Selecting Fitting Parameters

As the number of fitting parameters increases it may be necessary to only refine a subset of the parameters.  In MDMC this can be done in two ways:

## Fixing a Parameter

`Parameter` objects can be fixed.  This can either be set when they are initialised (created) or changed for an existing `Parameter`.  By default, an initialised `Parameter` is **not** fixed.

In [1]:
from MDMC.MD.parameters import Parameter
charge = Parameter(value=0.5, name='charge', fixed=True, unit='e')

# Trying to set the value of a fixed Parameter will result in a warning...
charge.value = 12

In [2]:
# ... and the value will remain unchanged
print(charge.value)

0.5 e


In [3]:
# Existing Parameters can also be fixed
sigma = Parameter(1.0, name='sigma', unit='Ang')
print('Is sigma fixed: {}'.format(sigma.fixed))
sigma.fixed = True
print('Is sigma fixed: {}'.format(sigma.fixed))

Is sigma fixed: False
Is sigma fixed: True


## Passing a subset of Parameters to Control

Only parameters that are passed to `Control` (as `fit_parameters`) will be refined.  While it is simplest to pass all parameters in a `Universe` to `Control`, it is also possible to filter out a subset.  To demonstrate this, a universe filled with SPCE water molecules is read from [Building a Universe](building-a-universe.ipynb).

The `%%capture` and `%run` commands below simply executes the [Building a Universe](building-a-universe.ipynb) notebook and captures the variables into this notebook. They are only valid if they are executed in the same folder as the [Building a Universe](building-a-universe.ipynb) notebook. Otherwise, please copy the last section of [Building a Universe](building-a-universe.ipynb) to set the same state.

In [4]:
%%capture
# Run Building a universe notebook and hide output
%run "building-a-universe.ipynb"

There are 8 parameters in the universe:

In [5]:
print(universe.parameters)

{'equilibrium_state (#1372)': <Parameter
 {ID: 1372,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#1373)': <Parameter
 {ID: 1373,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#1078)': <Parameter
 {ID: 1078,
  type: 'equilibrium_state',
  value: 109.47 deg,
  unit: 'deg',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#1079)': <Parameter
 {ID: 1079,
  type: 'potential_strength',
  value: 383.0 kJ / mol rad ^ 2,
  unit: 'kJ / mol rad ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential'

So while all 8 parameters can be passed when initiliasing `Control`, they can also be filtered. `Parameters` objects have a number of convenience methods to assist with this:

In [6]:
parameters = universe.parameters
help(parameters)

Help on Parameters in module MDMC.MD.parameters object:

class Parameters(builtins.dict)
 |  Parameters(init_parameters: "Optional[Union[Parameter, 'list[Parameter]']]" = None)
 |  
 |  A `dict-like` object where every element is a ``Parameter`` indexed by name,
 |  which contains a number of helper methods for filtering.
 |  
 |  Although ``Parameters`` is a `dict`, it should be treated like a `list` when writing to it;
 |  i.e. initialise it using a `list` and use `append` to add to it. These parameters can
 |  then be accessed by their name as a key.
 |  
 |  In short; Parameters writes like a list and reads like a dict.
 |  
 |  Parameters
 |  ----------
 |  init_parameters: ``Parameter`` or `list` of ``Parameter``s, optional, default None
 |      The initial ``Parameter`` objects that the ``Parameters`` object contains.
 |  
 |  Attributes
 |  ----------
 |  array: np.ndarray
 |      An alphabetically-sorted numpy array of the ``Parameter``s stored in this object.
 |  
 |  Method 

For example, if only the charge parameters should be refined, `parameters` could be filtered by name:

In [7]:
charges = parameters.filter_name('charge')
print(charges)

# charges is a Parameters object
print('\nThe class of charges is: {}'.format(type(charges)))

{'charge (#2144)': <Parameter
 {ID: 2144,
  type: 'charge',
  value: -0.8476 e,
  unit: 'e',
  fixed: False,
  constraints: None,
  interactions_name: 'Coulombic',
  functions_name: 'Coulomb',
  tied: False}>, 'charge (#1457)': <Parameter
 {ID: 1457,
  type: 'charge',
  value: 0.4238 e,
  unit: 'e',
  fixed: False,
  constraints: None,
  interactions_name: 'Coulombic',
  functions_name: 'Coulomb',
  tied: False}>}

The class of charges is: <class 'MDMC.MD.parameters.Parameters'>


As each filter returns a `Parameters` object, filters can be chained together. For example, to find the potential strengths of all bonds:

In [8]:
bond_potential_strengths = parameters.filter_name('potential_strength').filter_interaction('Bond')
print(bond_potential_strengths)

# These operations are commutative
print('\nThe order these methods are'
      ' applied does not matter: {}'.format(bond_potential_strengths
                                             == parameters.filter_interaction('Bond').filter_name('potential_strength')))

{'potential_strength (#1373)': <Parameter
 {ID: 1373,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#1301)': <Parameter
 {ID: 1301,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#1693)': <Parameter
 {ID: 1693,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#6277)': <Parameter
 {ID: 6277,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_n

It is also possible to filter parameters based on the properties of the atoms to which they apply.  For instance, we can filter the SPCE parameters so that only parameters of interactions on H atoms are shown:

In [9]:
H_parameters = parameters.filter_atom_attribute('name', 'H')
print(H_parameters)

{'equilibrium_state (#1372)': <Parameter
 {ID: 1372,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#1373)': <Parameter
 {ID: 1373,
  type: 'potential_strength',
  value: 4637.0 kJ / mol Ang ^ 2,
  unit: 'kJ / mol Ang ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#1078)': <Parameter
 {ID: 1078,
  type: 'equilibrium_state',
  value: 109.47 deg,
  unit: 'deg',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'potential_strength (#1079)': <Parameter
 {ID: 1079,
  type: 'potential_strength',
  value: 383.0 kJ / mol rad ^ 2,
  unit: 'kJ / mol rad ^ 2',
  fixed: False,
  constraints: None,
  interactions_name: 'BondAngle',
  functions_name: 'HarmonicPotential'

Finally there is also a more flexible method (`Parameters.filter`) which can be used in conjunction with any function to filter the parameters:

In [10]:
def is_length(parameter):
    if parameter.unit == 'Ang':
        return True
    else:
        return False
length_parameters = parameters.filter(is_length)
print(length_parameters)

# For those more familiar with Python, this can also be done using a lambda
lambda_length_parameters = parameters.filter(lambda x: x.unit == 'Ang')
print('\nThe same filter can be achieved using lambdas: {}'.format(lambda_length_parameters == length_parameters))

{'equilibrium_state (#1372)': <Parameter
 {ID: 1372,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#1300)': <Parameter
 {ID: 1300,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#1692)': <Parameter
 {ID: 1692,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#6276)': <Parameter
 {ID: 6276,
  type: 'equilibrium_state',
  value: 1.0 Ang,
  unit: 'Ang',
  fixed: False,
  constraints: None,
  interactions_name: 'Bond',
  functions_name: 'HarmonicPotential',
  tied: False}>, 'equilibrium_state (#2012)': <Parameter
 {ID: 2012,
  t